<h1><i>Importação de Bibliotecas</i><h1>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
from urllib.parse import urljoin

---

<h1><i>Configuração de Sessão</i></h1>

In [2]:
# Definindo o User-Agent para a sessão
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0'
}

# OPERA = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0'
# EDGE = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'

In [3]:
# URL para a página de login
login_url = 'https://ppi.gov.br/acesso/'
# Configuração inicial da sessão de requisições com login
session = requests.Session()
session.headers.update(headers)
session.verify = False

In [4]:
# Acessar a página de login para obter campos ocultos e CSRF
response = session.get(login_url, verify=False)
soup = BeautifulSoup(response.content, 'html.parser')
# Obter campos ocultos para incluir na solicitação de login
hidden_inputs = {tag['name']: tag.get('value', '') for tag in soup.find_all('input', type='hidden')}

c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
payload = {
    'log': 'Lucas',  # Nome de usuário
    'pwd': 'PrPPI2024',  # Senha
    **hidden_inputs  # Incluir campos ocultos
    }

In [6]:
# Tentar fazer o login
login_response = session.post(login_url, data=payload, verify=False)

# Verificar se o login foi bem-sucedido
if "Perfil" in login_response.text or "perfil" in login_response.url:
    print("Login efetuado com sucesso.")
else:
    print("Falha no login. Verifique credenciais e URL.")
    print("Detalhes da resposta:", login_response.text[:1000])  # Para diagnóstico

c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Login efetuado com sucesso.


---

<h1><i>Definição de Funções</i></h1>

In [7]:
def get_text_or_none(soup, selector, attr_name=None, attr_value=None):
    # Encontrar o elemento baseado no seletor e atributos
    if attr_name and attr_value:
        element = soup.find(selector, {attr_name: attr_value})
    else:
        element = soup.find(selector)
    
    # Retornar o valor do elemento se encontrado, senão None
    return element['value'] if element and 'value' in element.attrs else None

In [8]:
def get_checked_checkboxes(soup, name):
    checked_boxes = soup.find_all('input', {'type': 'checkbox', 'name': name})
    checked_labels = []
    for box in checked_boxes:
        if box.has_attr('checked'):
            # Encontrar o label associado ao checkbox marcado
            label = box.find_parent('label')
            if label:
                checked_labels.append(label.get_text(strip=True))
    return checked_labels

In [9]:
# Função para obter opções selecionadas de um select de forma segura
def get_selected_options_safe(soup, select_id):
    select = soup.find('select', {'id': select_id})
    if select is not None:
        return [(option['value'], option.text.strip()) for option in select.find_all('option', selected=True)]
    else:
        return []

In [10]:
def get_select_option(soup, field_name):
    select_element = soup.find('select', {'name': field_name})
    if select_element:
        selected_option = select_element.find('option', selected=True)
        if selected_option:
            return selected_option.get_text(strip=True)
    return None

In [11]:
def extract_etapas(soup):
    etapas = []
    for i in range(6):  # Assumindo que há 6 etapas como discutido
        etapa_info = {}
        radio_checked = soup.find('input', {'name': f'acf[field_6310f4ec9b055][row-{i}][field_6311023434b2e]', 'checked': True})
        if radio_checked:
            etapa_info['etapa'] = radio_checked.find_next_sibling('span').text.strip()
        
        # Extrair o status selecionado
        status_select = soup.find('select', {'id': f'acf-field_6310f4ec9b055-row-{i}-field_631112a34b7f6'})
        if status_select:
            selected_option = status_select.find('option', selected=lambda x: x is not None)
            etapa_info['status'] = selected_option.text.strip() if selected_option else ""
        else:
            etapa_info['status'] = ""

        # Extrair datas de início e fim
        start_date_input = soup.find('input', {'id': f'acf-field_6310f4ec9b055-row-{i}-field_6311094d8fae5'})
        etapa_info['data_inicio'] = start_date_input['value'] if start_date_input else ""

        end_date_input = soup.find('input', {'id': f'acf-field_6310f4ec9b055-row-{i}-field_631109a88fae7'})
        etapa_info['data_fim'] = end_date_input['value'] if end_date_input else ""

        etapas.append(etapa_info)
    return etapas

In [12]:
# Função para extrair reuniões
def get_reunions(soup):
    reunions = []
    reunion_fields = soup.find_all('td', {'class': 'acf-field-text'})
    for field in reunion_fields:
        input_element = field.find('input', {'type': 'text'})
        if input_element and input_element.has_attr('value'):
            reunions.append(input_element['value'])
    return reunions

In [13]:
# Função para Extrair a Data de Qualificação
def get_date_field(soup, field_name):
    date_element = soup.find('input', {'name': field_name})
    if date_element and 'value' in date_element.attrs:
        return date_element['value']
    return None

In [14]:
# Função para Extrair Ano do Leilão
def get_number_field(soup, field_name):
    number_element = soup.find('input', {'name': field_name})
    if number_element and 'value' in number_element.attrs:
        return number_element['value']
    return None

In [15]:
# Função para extrair o link do projeto
def get_project_link(soup):
    link_element = soup.find('span', {'id': 'sample-permalink'}).find('a')
    if link_element and 'href' in link_element.attrs:
        return link_element['href']
    return None

In [16]:
# Função para extrair campos de texto por ID
def get_text_field_by_id(soup, field_id):
    element = soup.find('input', {'id': field_id})
    if element and 'value' in element.attrs:
        return element['value']
    return None

---

<h1><i>Processamento de URLs</i></h1>

In [17]:
def fetch_project_links(base_url, max_pages=100):
    project_urls = []
    last_valid_url = base_url
    for page in tqdm(range(1, max_pages + 1), desc="Extraindo links"):
        page_url = f"{base_url}&paged={page}"
        response = session.get(page_url, allow_redirects=True)
        
        # Verifica se a página foi redirecionada para a última URL válida
        if response.url == last_valid_url:
            print("Nenhuma página nova, parando a busca.")
            break
        else:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = soup.find_all('a', href=True)
            for link in links:
                if '/post.php?post=' in link['href'] and '&action=edit' in link['href']:
                    full_url = urljoin(base_url, link['href'])
                    if full_url not in project_urls:
                        project_urls.append(full_url)
            last_valid_url = response.url  # Atualiza o último URL válido após processar a página
    return project_urls

In [18]:
# Iniciar a extração
base_url = 'https://ppi.gov.br/wp-admin/edit.php?post_status=publish&post_type=projetos'
project_urls = fetch_project_links(base_url)
print(f"Total de links coletados: {len(project_urls)}")

# Salvar os links em um arquivo
with open('project_urls.txt', 'w') as file:
    for url in project_urls:
        file.write(url + '\n')

Extraindo links:   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Extraindo links:   1%|          | 1/100 [00:01<03:11,  1.93s/it]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Extraindo links:   2%|▏         | 2/100 [00:03<03:13,  1.97s/it]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureReq

Nenhuma página nova, parando a busca.
Total de links coletados: 639


---

<h1><i>Extração dos Dados</i></h1>

In [19]:
data_list = []  # Lista para armazenar os dados coletados# Processar cada URL do projeto

for url in tqdm(project_urls, desc='Extraindo dados dos projetos'):
    response = session.get(url)
    if response.status_code != 200:
        print(f"Falha ao acessar {url}, Status Code: {response.status_code}")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find('input', {'id': 'title'}).get('value')
    description = soup.find('textarea', {'id': 'content'}).text
    status = get_checked_checkboxes(soup, 'tax_input[status][]')
    uf = get_checked_checkboxes(soup, 'tax_input[uf][]')
    setores = get_checked_checkboxes(soup, 'tax_input[setores][]')
    ods = get_checked_checkboxes(soup, 'tax_input[ods][]')
    id_projeto = get_text_or_none(soup, 'input', 'name', 'acf[field_636a4fbba8b53]')
    ministerio = get_text_or_none(soup, 'input', 'name', 'acf[field_636a91aa285db]')
    secretaria = soup.find('input', {'id': 'acf-field_636a91df285dd'}).get('value')
    resolucoes = get_selected_options_safe(soup, 'acf-field_6388d4eae3d87')
    decretos = get_selected_options_safe(soup, 'acf-field_6388d5f03c024')
    etapas = extract_etapas(soup)
    reunions = get_reunions(soup)
    data_qualificacao = get_date_field(soup, 'acf[field_636a8610ce9ae]')
    modalidade_operacional = get_select_option(soup, 'acf[field_636a89fd5a3cf]')
    detalhamento_modalidade_operacional = get_select_option(soup, 'acf[field_636a8a70b765c]')
    extensao = get_text_field_by_id(soup, 'acf-field_64516f9fa08fb')
    capex = get_text_or_none(soup, 'input', 'name', 'acf[field_63231967a8b12]')
    leilao = get_select_option(soup, 'acf[field_636aa61ecda34]')
    ano_leilao = get_number_field(soup, 'acf[field_64bec474d7c03]')
    projeto_ativo = get_select_option(soup, 'acf[field_636a98a70c7a4]')
    permalink = get_project_link(soup)
    
    
    project_data = {
        'link': permalink,
        'title': title,
        'status': status,
        'uf': uf,
        'setores': setores,
        'ods': ods,
        'identificador_do_projeto': id_projeto,
        'ministerio': ministerio,
        'secretaria': secretaria,
        'resolucoes': resolucoes,
        'decretos': decretos,
        'etapas': etapas,
        'reuniao': reunions,
        'data_qualificacao': data_qualificacao,
        'modalidade_operacional': modalidade_operacional,
        'detalhamento_modalidade_operacional': detalhamento_modalidade_operacional,
        'extensão': extensao,
        'capex': capex,
        'leilao': leilao,
        'ano_leilao': ano_leilao,
        'projeto_ativo': projeto_ativo
    }
    
    data_list.append(project_data)

Extraindo dados dos projetos:   0%|          | 0/639 [00:00<?, ?it/s]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Extraindo dados dos projetos:   0%|          | 1/639 [00:01<16:15,  1.53s/it]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.1.101.101'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Extraindo dados dos projetos:   0%|          | 2/639 [00:03<19:08,  1.80s/it]c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python313\Lib\site-packages\urll

---

<h1><i>Salvamento dos Dados</i></h1>

In [20]:
# Salvar os dados coletados em um arquivo JSON
with open('project_data.json', 'w', encoding='utf-8') as file:
    json.dump(data_list, file, ensure_ascii=False, indent=4)

In [21]:
# Fazer uma cópia do arquivo JSON diretamente para o caminho desejado
shutil.copy('project_data.json', r'C:\PR1049457\lucas.bruno\Downloads\Projects\PPI.GOV\Pandas\project_data_copy.json')

'C:\\PR1049457\\lucas.bruno\\Downloads\\Projects\\PPI.GOV\\Pandas\\project_data_copy.json'

---

<h1><i>SCRAP PPI.GOV - PROJETOS</i></h1>
<h1><i>Importação de Bibliotecas</i></h1>
<p>O script importa diversas bibliotecas necessárias para o processo de scraping e manipulação dos dados:</p>
<pre>
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
from urllib.parse import urljoin
</pre>
<p>Essas bibliotecas são utilizadas para requisições HTTP, parsing de HTML, manipulação de JSON, monitoramento do progresso e operações em arquivos e URLs.</p>

<h1><i>Configuração de Sessão</i></h1>
<p>A seguir, o código configura a sessão de requisição HTTP:</p>
<pre>
# Definindo o User-Agent para a sessão
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0'
}
# URL para a página de login
login_url = 'https://ppi.gov.br/acesso/'
session = requests.Session()
session.headers.update(headers)
response = session.get(login_url)
</pre>
<p>Após definir o <code>User-Agent</code>, o script inicializa uma sessão e faz uma requisição para a página de login para buscar campos ocultos necessários para realizar o login.</p>

<h1><i>Definição de Funções</i></h1>
<p>O script define várias funções para extrair dados de forma eficiente da página HTML:</p>
<ul>
<li><b><i>get_text_or_none</i></b>: Extrai o valor de um campo HTML, caso exista.</li>
<li><b><i>get_checked_checkboxes</i></b>: Extrai os valores de checkboxes marcados.</li>
<li><b><i>get_selected_options_safe</i></b>: Extrai as opções selecionadas de um campo <code>select</code> de forma segura.</li>
<li><b><i>get_select_option</i></b>: Extrai o valor selecionado de um campo <code>select</code>.</li>
<li><b><i>extract_etapas</i></b>: Extrai as etapas de um projeto, considerando as entradas de tipo radio button.</li>
<li><b><i>get_reunions</i></b>: Extrai os valores de reuniões cadastradas em campos de texto.</li>
<li><b><i>get_date_field</i></b>: Extrai uma data de um campo específico.</li>
<li><b><i>get_number_field</i></b>: Extrai um valor numérico de um campo.</li>
<li><b><i>get_project_link</i></b>: Extrai o link permanente do projeto.</li>
</ul>

<h1><i>Processamento de URLs</i></h1>
<p>O script então processa URLs de projetos a partir de uma página principal de listagem de projetos, coletando os links para cada projeto individualmente:</p>
<pre>
def fetch_project_links(base_url, max_pages=100):
project_urls = []
last_valid_url = base_url
for page in tqdm(range(1, max_pages + 1), desc="Extraindo links"):
page_url = f"{base_url}&paged={page}"
response = session.get(page_url, allow_redirects=True)
if response.url == last_valid_url:
print("Nenhuma página nova, parando a busca.")
break
else:
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all('a', href=True)
for link in links:
if '/post.php?post=' in link['href'] and '&action=edit' in link['href']:
    full_url = urljoin(base_url, link['href'])
    if full_url not in project_urls:
        project_urls.append(full_url)
last_valid_url = response.url
return project_urls
</pre>
<p>O script extrai os links dos projetos através de um loop nas páginas de resultados, verificando se há novos links e coletando-os.</p>

<h1><i>Extração dos Dados</i></h1>
<p>A coleta dos dados de cada projeto é feita por meio de requisições à URL de cada projeto individualmente:</p>
<pre>
data_list = []  # Lista para armazenar os dados coletados

for url in tqdm(project_urls, desc='Extraindo dados dos projetos'):
response = session.get(url)
if response.status_code != 200:
print(f"Falha ao acessar {url}, Status Code: {response.status_code}")
continue
soup = BeautifulSoup(response.content, 'html.parser')

title = soup.find('input', {'id': 'title'}).get('value')
description = soup.find('textarea', {'id': 'content'}).text
status = get_checked_checkboxes(soup, 'tax_input[status][]')
uf = get_checked_checkboxes(soup, 'tax_input[uf][]')
...

project_data = {
'link': permalink,
'title': title,
'status': status,
'uf': uf,
'setores': setores,
...
}

data_list.append(project_data)
</pre>
<p>Para cada URL de projeto, o script realiza uma requisição e utiliza as funções definidas anteriormente para extrair diversas informações sobre o projeto, como título, descrição, status, e etapas, e armazena os dados coletados em uma lista.</p>

<h1><i>Salvamento dos Dados</i></h1>
<p>Após a extração dos dados, o script salva as informações em um arquivo JSON:</p>
<pre>
with open('project_data.json', 'w', encoding='utf-8') as file:
json.dump(data_list, file, ensure_ascii=False, indent=4)
</pre>
<p>Além disso, o script faz uma cópia do arquivo JSON:</p>
<pre>
shutil.copy('project_data.json', 'project_data_copy.json')
</pre>
<p>Esses arquivos JSON contêm todos os dados extraídos, sendo a cópia criada para fins de backup ou processamento adicional.</p>

<h2><i>Conclusão</i></h2>
<p>Este script realiza a coleta de dados estruturados de projetos de um site específico, utilizando técnicas de scraping e armazenamento seguro dos dados extraídos em um arquivo JSON. Ele também garante a integridade dos dados através de cópias de segurança.</p>

---